In [1]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4264  100  4264    0     0  47377      0 --:--:-- --:--:-- --:--:-- 47377
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
Uninstalling torch-1.5.0+cu101:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.5.0+cu101
Uninstalling torchvision-0.6.0+cu101:
  Successfully uninstalled torchvision-0.6.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 90.6 MiB/ 90.6 MiB]                                                
Operation completed over 1 objects/90.6 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly-cp36-cp36m-linux_x86_64.whl...
/ [1 files][121.4 MiB/121.4 MiB]                                                
Operation completed over 1 objects/121.4 MiB.         

In [0]:
!export XLA_USE_BF16=1

In [3]:
!pip install transformers

     |████████████████████████████████| 675kB 3.4MB/s 
     |████████████████████████████████| 3.8MB 19.0MB/s 
     |████████████████████████████████| 890kB 32.9MB/s 
     |████████████████████████████████| 1.1MB 15.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=6070f305ce94484f0d233c77f7860d700a9a35a384c858fb4d70c6c486a5ea8f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
import os
os.chdir("drive/My Drive/kaggle-tweet-sentiment-extraction/alberta")
!pwd

/content/drive/My Drive/kaggle-tweet-sentiment-extraction/alberta


In [0]:
import transformers

In [19]:
tokenizer = transformers.AlbertTokenizer.from_pretrained("albert-xlarge-v2")

In [0]:
new_tokens = ['<', '>', '@', '\\', '^', '_', '`', '{', '|', '}', '~', '¿', '⁄']
new_tokens2 =['▁<', '▁>', '▁@', '▁\\', '▁^', '▁_', '▁`', '▁{', '▁|', '▁}', '▁~', '▁¿', '▁⁄']

In [0]:
num_added_toks = tokenizer.add_tokens(new_tokens+new_tokens2)

In [0]:
num_added_toks

26

In [0]:
import sentencepiece_pb2
import sentencepiece as spm

class SentencePieceTokenizer:
    def __init__(self, model_path):
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(model_path)
    
    def encode(self, sentence):
        spt = sentencepiece_pb2.SentencePieceText()
        spt.ParseFromString(self.sp.encode_as_serialized_proto(sentence))
        offsets = []
        token_ids = []
        tokens = []
        for piece in spt.pieces:
            token_ids.append(piece.id)
            offsets.append((piece.begin, piece.end))
            tokens.append(piece.surface)
        return token_ids, offsets, tokens

In [0]:
def find_start_and_end(tweet, selected_text):
    len_st = len(selected_text)
    start = None
    end = None
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[0]):
        if tweet[ind: ind+len_st] == selected_text:
            start = ind
            end = ind + len_st - 1
            break
    return start, end

def create_offsets(tokens):
    offsets = []
    off1 = 0
    off2 = 0
    first = True
    for token in tokens:
        if '▁' in token:
            ntoken = token.replace('▁','')
            if not first:
                off1 = off1 + 1
            off2 = off1+len(ntoken)
            offsets.append((off1,off2))
            off1 = off2  
            first = False
        else:
            off2 = off1+len(token)
            offsets.append((off1,off2))
            off1 = off2
            
    return offsets

def process_with_offsets(args, tweet, selected_text, sentiment, tokenizer, sptokenizer):

    tweet = " ".join(str(tweet).split())
    selected_text = " ".join(str(selected_text).split())

    start_index, end_index = find_start_and_end(tweet, selected_text)
    
    print(start_index, end_index)

    char_targets = [0]*len(tweet)
    if start_index != None and end_index != None:
        for ct in range(start_index, end_index+1):
            char_targets[ct] = 1
            
    #print(char_targets)
    
    encoded = tokenizer.encode_plus(
                    sentiment,
                    tweet,
                    max_length=args.max_seq_len,
                    pad_to_max_length=True,
                    return_token_type_ids=True,
                    #return_offsets_mapping=True,
                
                )
    
    _, offsets, _ = sptokenizer.encode(tweet)
    tweet_offsets = [(0,0)]*3 + offsets + [(0,0)]


    padding_length = args.max_seq_len - len(tweet_offsets)
    if padding_length > 0:
        tweet_offsets = tweet_offsets + ([(0,0)] * padding_length)
    
    encoded["offset_mapping"] = tweet_offsets
    
    #print(tweet_offsets)
    #print(len(tweet_offsets))

    target_idx = []
    for j, (offset1, offset2) in enumerate(encoded["offset_mapping"]):
        if j > 2:
            if sum(char_targets[offset1:offset2]) > 0:
                target_idx.append(j)

    encoded["start_position"] = target_idx[0]
    encoded["end_position"] = target_idx[-1]
    encoded["tweet"] = tweet
    encoded["selected_text"] = selected_text
    encoded["sentiment"] = sentiment

    return encoded

In [0]:
#tokenizer = transformers.AlbertTokenizer.from_pretrained("albert-base-v1")
class args:
    max_seq_len = 192
    sp_path = "cs224n-squad2.0-albert-large-v2/spiece.model"

In [0]:
import pandas as pd

In [0]:
train_df = pd.read_csv("../input/train.csv")

In [0]:
sptokenizer = SentencePieceTokenizer(args.sp_path)

In [41]:
for i , (textID, text, selected_text, sentiment) in enumerate(train_df.values):
    
    
    features = process_with_offsets(args, text, selected_text, sentiment, tokenizer,sptokenizer )
    
    print(features["tweet"])
    print(features["selected_text"])
    start = features["start_position"]
    end = features["end_position"]
    
    print(start, end)
    
    selected_inputs = features["input_ids"][start:end+1]
    
    #print(tokenizer.decode(selected_inputs))
    
    #print(features["input_ids"])

    
    offsets = features["offset_mapping"]

    print(features["tweet"][offsets[start][0]:offsets[end][-1]+1])
    
    
    
    print("="*50)
    
    if i == 200:
        break
    

0 34
I`d have responded, if I were going
I`d have responded, if I were going
4 13
I`d have responded, if I were going
0 7
Sooo SAD I will miss you here in San Diego!!!
Sooo SAD
4 8
Sooo SAD 
11 21
my boss is bullying me...
bullying me
6 8
 bullying me.
16 29
what interview! leave me alone
leave me alone
6 8
 leave me alone
0 12
Sons of ****, why couldn`t they put them on the releases we already bought
Sons of ****,
4 8
Sons of ****, 
0 91
http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth
http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth
3 32
http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth
30 32
2am feedings for the baby are fun when he is all smiles and coos
fun
11 11
 fun 
0 9
Soooo high
Soooo high
4 6
Soooo high
0 10
Both of you
Both of you
4 8
Both of you
10 37
Journey!? Wow... u just became cooler. hehe... (is that possible!?)
Wow... u just became coole

In [0]:
tokenizer.encode("______")

[2, 30005, 30005, 30005, 30005, 30005, 30005, 3]

In [0]:
tokenizer.tokenize("______")

['_', '_', '_', '_', '_', '_']

In [0]:
tokenizer.decode([2, 30005, 30005, 30005, 30005, 30005, 30005, 3])

'[CLS] _ _ _ _ _ _ [SEP]'

In [0]:
tokenizer.get_vocab()

{'<pad>': 0,
 '<unk>': 1,
 '[CLS]': 2,
 '[SEP]': 3,
 '[MASK]': 4,
 '(': 5,
 ')': 6,
 '"': 7,
 '-': 8,
 '.': 9,
 '–': 10,
 '£': 11,
 '€': 12,
 '▁': 13,
 '▁the': 14,
 ',': 15,
 '▁of': 16,
 '▁and': 17,
 's': 18,
 '▁in': 19,
 '▁to': 20,
 '▁a': 21,
 "'": 22,
 '▁was': 23,
 '▁he': 24,
 '▁is': 25,
 '▁for': 26,
 '▁on': 27,
 '▁as': 28,
 '▁with': 29,
 '▁that': 30,
 '▁i': 31,
 '▁it': 32,
 '▁his': 33,
 '▁by': 34,
 '▁at': 35,
 '▁her': 36,
 '▁from': 37,
 't': 38,
 '▁she': 39,
 '▁an': 40,
 '▁had': 41,
 '▁you': 42,
 'd': 43,
 '▁be': 44,
 ':': 45,
 '▁were': 46,
 '▁but': 47,
 '▁this': 48,
 'i': 49,
 '▁are': 50,
 '▁my': 51,
 '▁not': 52,
 '▁one': 53,
 '▁or': 54,
 '▁me': 55,
 '▁which': 56,
 '▁have': 57,
 'a': 58,
 '▁they': 59,
 '?': 60,
 '▁him': 61,
 'e': 62,
 '▁has': 63,
 '▁first': 64,
 '▁all': 65,
 '▁their': 66,
 '▁also': 67,
 'ing': 68,
 'ed': 69,
 '▁out': 70,
 '▁up': 71,
 '▁who': 72,
 ';': 73,
 '▁been': 74,
 '▁after': 75,
 '▁when': 76,
 '▁into': 77,
 '▁new': 78,
 'm': 79,
 '▁there': 80,
 '▁two': 81,
 '▁

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="albert-base-v1" \
                  --output_dir="../weights/albert-base-v1" \
                  --exp_name="albert-base-v1_seed42" \
                  --batch_size=64 \
                  --learning_rate=2e-5 \
                  --seed=42

2020-05-30 13:34:05.004193: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Training is Starting ...... 
Train E:1 - Loss:2.70 - acc1:0.60 - acc2:0.61: 100% 43/43 [04:40<00:00,  6.53s/it]
Eval E:1 - Loss:1.83 - Jac:0.67: 100% 11/11 [01:26<00:00,  7.85s/it]
**** Epoch 1 **==>** Jaccard = 0.6729363716036827
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.76 - acc1:0.68 - acc2:0.70: 100% 43/43 [02:02<00:00,  2.86s/it]
Eval E:2 - Loss:1.71 - Jac:0.68: 100% 11/11 [00:29<00:00,  2.70s/it]
**** Epoch 2 **==>** Jaccard = 0.6835742473101187
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.60 - acc1:0.71 - acc2:0.73: 100% 43/43 [01:59<00:00,  2.77s/it]
Eval E:3 - Loss:1.71 - Jac:0.69: 100% 11/11 [00:29<00:00,  2.65s/it]
**** Epoch 3 **==>** Jaccard = 0.6893138748567152
**** Model Improved !!!! Saving Model
Train E:4 - Loss:1.49 - acc1:0.72 - acc2:0.74: 100% 43/43 [02:02<00:00,  2.84s/it]
Eval E:4 - Loss:1.74 -

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="albert-base-v2" \
                  --output_dir="../weights/albert-base-v2" \
                  --exp_name="albert-base-v2_seed42" \
                  --batch_size=64 \
                  --learning_rate=2e-5 \
                  --seed=42

2020-05-30 13:52:00.390551: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 684/684 [00:00<00:00, 526kB/s]
Downloading: 100% 760k/760k [00:00<00:00, 4.98MB/s]
Downloading: 100% 47.4M/47.4M [00:02<00:00, 22.8MB/s]
Training is Starting ...... 
Train E:1 - Loss:3.10 - acc1:0.57 - acc2:0.55: 100% 43/43 [02:54<00:00,  4.06s/it]
Eval E:1 - Loss:1.95 - Jac:0.66: 100% 11/11 [01:14<00:00,  6.78s/it]
**** Epoch 1 **==>** Jaccard = 0.6617361853751926
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.79 - acc1:0.69 - acc2:0.70: 100% 43/43 [01:41<00:00,  2.35s/it]
Eval E:2 - Loss:1.74 - Jac:0.68: 100% 11/11 [00:30<00:00,  2.76s/it]
**** Epoch 2 **==>** Jaccard = 0.6773509693072595
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.57 - acc1:0.71 - acc2:0.73: 100% 43/43 [01:40<00:00,  2.34s/it]
Eval E:3 - Loss:1.72 - Jac:0.69: 100% 11/11 [00:30<00:00,  2.77s/it]
**** Epoch 3 **==>** Jaccard = 0.68505

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="albert-large-v1" \
                  --output_dir="../weights/albert-large-v1" \
                  --exp_name="albert-large-v1_seed42" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42

2020-05-30 14:07:50.273786: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 685/685 [00:00<00:00, 565kB/s]
Downloading: 100% 760k/760k [00:00<00:00, 4.94MB/s]
Downloading: 100% 71.5M/71.5M [00:01<00:00, 42.2MB/s]
Training is Starting ...... 
Train E:1 - Loss:2.73 - acc1:0.60 - acc2:0.60: 100% 86/86 [12:06<00:00,  8.45s/it]
Eval E:1 - Loss:1.86 - Jac:0.66: 100% 22/22 [03:03<00:00,  8.35s/it]
**** Epoch 1 **==>** Jaccard = 0.6565906154928637
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.77 - acc1:0.69 - acc2:0.71: 100% 86/86 [05:56<00:00,  4.15s/it]
Eval E:2 - Loss:1.74 - Jac:0.68: 100% 22/22 [01:06<00:00,  3.03s/it]
**** Epoch 2 **==>** Jaccard = 0.6778138273632333
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.62 - acc1:0.71 - acc2:0.72: 100% 86/86 [06:01<00:00,  4.20s/it]
Eval E:3 - Loss:1.75 - Jac:0.68: 100% 22/22 [01:06<00:00,  3.00s/it]
**** Epoch 3 **==>** Jaccard = 0.67500

In [0]:
transformers.AlbertForQuestionAnswering.from_pretrained("elgeish/cs224n-squad2.0-albert-base-v2")

AlbertForQuestionAnswering(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="elgeish/cs224n-squad2.0-albert-base-v2" \
                  --output_dir="../weights/albert-xlarge-v2-squad-v2" \
                  --exp_name="albert-xlarge-v2_seed42" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:2.18 - acc1:0.65 - acc2:0.66: 100% 86/86 [04:03<00:00,  2.83s/it]
Eval E:1 - Loss:1.70 - Jac:0.68: 100% 22/22 [01:15<00:00,  3.41s/it]
**** Epoch 1 **==>** Jaccard = 0.6759064830608736
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.58 - acc1:0.71 - acc2:0.73: 100% 86/86 [02:59<00:00,  2.09s/it]
Eval E:2 - Loss:1.78 - Jac:0.68: 100% 22/22 [00:44<00:00,  2.03s/it]
**** Epoch 2 **==>** Jaccard = 0.6810433447354576
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.48 - acc1:0.72 - acc2:0.74: 100% 86/86 [02:57<00:00,  2.06s/it]
Eval E:3 - Loss:1.80 - Jac:0.68: 100% 22/22 [00:45<00:00,  2.09s/it]
**** Epoch 3 **==>** Jaccard = 0.6816985607750776
**** Model Improved !!!! Saving Model
Train E:4 - Loss:1.26 - acc1

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-xlarge-v2-squad-v2" \
                  --exp_name="albert-xlarge-v2_seed42" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42

Downloading: 100% 700/700 [00:00<00:00, 484kB/s]
Downloading: 100% 760k/760k [00:00<00:00, 5.07MB/s]
Downloading: 100% 156/156 [00:00<00:00, 112kB/s]
Downloading: 100% 39.0/39.0 [00:00<00:00, 27.6kB/s]
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Downloading: 100% 70.8M/70.8M [00:00<00:00, 70.8MB/s]
Training is Starting ...... 
Train E:1 - Loss:2.92 - acc1:0.58 - acc2:0.60: 100% 86/86 [07:33<00:00,  5.27s/it]
Eval E:1 - Loss:1.83 - Jac:0.67: 100% 22/22 [02:33<00:00,  6.97s/it]
**** Epoch 1 **==>** Jaccard = 0.6744359923648554
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.73 - acc1:0.70 - acc2:0.72: 100% 86/86 [04:32<00:00,  3.17s/it]
Eval E:2 - Loss:1.73 - Jac:0.68: 100% 22/22 [01:05<00:00,  2.96s/it]
**** Epoch 2 **==>** Jaccard = 0.6797768861000444
**** Model Improved !!!! Saving Model
Train E:3 - 

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-xlarge-v2-squad-v2" \
                  --exp_name="albert-xlarge-v2_seed42" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:2.92 - acc1:0.58 - acc2:0.60: 100% 86/86 [04:51<00:00,  3.39s/it]
Eval E:1 - Loss:1.83 - Jac:0.68: 100% 22/22 [01:31<00:00,  4.16s/it]
**** Epoch 1 **==>** Jaccard = 0.6847490222840185
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.73 - acc1:0.70 - acc2:0.72: 100% 86/86 [03:25<00:00,  2.39s/it]
Eval E:2 - Loss:1.73 - Jac:0.69: 100% 22/22 [00:44<00:00,  2.02s/it]
**** Epoch 2 **==>** Jaccard = 0.690907558353004
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.56 - acc1:0.72 - acc2:0.74: 100% 86/86 [03:25<00:00,  2.39s/it]
Eval E:3 - Loss:1.71 - Jac:0.70: 100% 22/22 [00:44<00:00,  2.00s/it]
**** Epoch 3 **==>** Jaccard = 0.6991612173425853
**** Model Improved !!!! Saving Model
Train E:4 - Loss:1.43 - acc1:

KeyboardInterrupt: ignored

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="elgeish/cs224n-squad2.0-albert-base-v2" \
                  --output_dir="../weights/albert-xlarge-v2-squad-v2" \
                  --exp_name="albert-xlarge-v2_seed42" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42

Downloading: 100% 699/699 [00:00<00:00, 522kB/s]
Downloading: 100% 760k/760k [00:00<00:00, 3.20MB/s]
Downloading: 100% 156/156 [00:00<00:00, 115kB/s]
Downloading: 100% 39.0/39.0 [00:00<00:00, 27.7kB/s]
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Downloading: 100% 46.7M/46.7M [00:00<00:00, 64.7MB/s]
Training is Starting ...... 
Train E:1 - Loss:2.18 - acc1:0.65 - acc2:0.66: 100% 86/86 [02:45<00:00,  1.92s/it]
Eval E:1 - Loss:1.70 - Jac:0.69: 100% 22/22 [01:02<00:00,  2.83s/it]
**** Epoch 1 **==>** Jaccard = 0.6895030548446159
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.58 - acc1:0.71 - acc2:0.73: 100% 86/86 [01:23<00:00,  1.03it/s]
Eval E:2 - Loss:1.78 - Jac:0.69: 100% 22/22 [00:18<00:00,  1.18it/s]
**** Epoch 2 **==>** Jaccard = 0.694456474487533
**** Model Improved !!!! Saving Model
Train E:3 - L

KeyboardInterrupt: ignored

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="albert-base-v2" \
                  --output_dir="../weights/albert-xlarge-v2-squad-v2" \
                  --exp_name="albert-xlarge-v2_seed42" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42

Downloading: 100% 684/684 [00:00<00:00, 549kB/s]
Downloading: 100% 760k/760k [00:00<00:00, 5.14MB/s]
Downloading: 100% 47.4M/47.4M [00:00<00:00, 59.9MB/s]
Training is Starting ...... 
Train E:1 - Loss:2.91 - acc1:0.56 - acc2:0.57: 100% 86/86 [01:30<00:00,  1.05s/it]
Eval E:1 - Loss:2.14 - Jac:0.65: 100% 22/22 [00:22<00:00,  1.04s/it]
**** Epoch 1 **==>** Jaccard = 0.6509608231241637
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.82 - acc1:0.68 - acc2:0.69: 100% 86/86 [01:23<00:00,  1.03it/s]
Eval E:2 - Loss:1.77 - Jac:0.69: 100% 22/22 [00:18<00:00,  1.17it/s]
**** Epoch 2 **==>** Jaccard = 0.6876017124690252
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.56 - acc1:0.71 - acc2:0.73: 100% 86/86 [01:24<00:00,  1.02it/s]
Eval E:3 - Loss:1.80 - Jac:0.69: 100% 22/22 [00:17<00:00,  1.23it/s]
**** Epoch 3 **==>** Jaccard = 0.6866600718207791
Train E:4 - Loss:1.38 - acc1:0.74 - acc2:0.76: 100% 86/86 [01:24<00:00,  1.02it/s]
Eval E:4 - Loss:1.93 - Jac:0.68: 100% 22/22 [00:18<

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="albert-base-v1" \
                  --output_dir="../weights/albert-xlarge-v2-squad-v2" \
                  --exp_name="albert-xlarge-v2_seed42" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42

Downloading: 100% 684/684 [00:00<00:00, 554kB/s]
Downloading: 100% 760k/760k [00:00<00:00, 4.99MB/s]
Downloading: 100% 47.4M/47.4M [00:01<00:00, 42.2MB/s]
Training is Starting ...... 
Train E:1 - Loss:2.31 - acc1:0.63 - acc2:0.65: 100% 86/86 [04:03<00:00,  2.84s/it]
Eval E:1 - Loss:1.77 - Jac:0.68: 100% 22/22 [01:16<00:00,  3.48s/it]
**** Epoch 1 **==>** Jaccard = 0.6770004717668282
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.69 - acc1:0.69 - acc2:0.71: 100% 86/86 [01:40<00:00,  1.16s/it]
Eval E:2 - Loss:1.72 - Jac:0.69: 100% 22/22 [00:18<00:00,  1.21it/s]
**** Epoch 2 **==>** Jaccard = 0.6911173188856166
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.53 - acc1:0.71 - acc2:0.73: 100% 86/86 [01:42<00:00,  1.19s/it]
Eval E:3 - Loss:1.73 - Jac:0.69: 100% 22/22 [00:17<00:00,  1.25it/s]
**** Epoch 3 **==>** Jaccard = 0.6910444520162462
Train E:4 - Loss:1.37 - acc1:0.74 - acc2:0.76: 100% 86/86 [01:40<00:00,  1.16s/it]
Eval E:4 - Loss:1.83 - Jac:0.69: 100% 22/22 [00:18<

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="albert-large-v2" \
                  --output_dir="../weights/albert-xlarge-v2-squad-v2" \
                  --exp_name="albert-xlarge-v2_seed42" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42

Downloading: 100% 685/685 [00:00<00:00, 546kB/s]
Downloading: 100% 760k/760k [00:00<00:00, 5.08MB/s]
Downloading: 100% 71.5M/71.5M [00:01<00:00, 45.6MB/s]
Training is Starting ...... 
Train E:1 - Loss:2.84 - acc1:0.59 - acc2:0.58: 100% 86/86 [03:33<00:00,  2.49s/it]
Eval E:1 - Loss:1.86 - Jac:0.68: 100% 22/22 [00:49<00:00,  2.25s/it]
**** Epoch 1 **==>** Jaccard = 0.6787875694516009
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.73 - acc1:0.70 - acc2:0.71: 100% 86/86 [03:24<00:00,  2.38s/it]
Eval E:2 - Loss:1.72 - Jac:0.70: 100% 22/22 [00:44<00:00,  2.02s/it]
**** Epoch 2 **==>** Jaccard = 0.6952532612162831
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.52 - acc1:0.72 - acc2:0.74: 100% 86/86 [03:24<00:00,  2.38s/it]
Eval E:3 - Loss:1.75 - Jac:0.69: 100% 22/22 [00:44<00:00,  2.01s/it]
**** Epoch 3 **==>** Jaccard = 0.6938119305451155
Train E:4 - Loss:1.37 - acc1:0.74 - acc2:0.76: 100% 86/86 [03:27<00:00,  2.41s/it]
Eval E:4 - Loss:1.80 - Jac:0.69: 100% 22/22 [00:45<

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="ktrapeznikov/albert-xlarge-v2-squad-v2" \
                  --output_dir="../weights/albert-xlarge-v2-squad-v2" \
                  --exp_name="albert-xlarge-v2_seed42" \
                  --batch_size=8 \
                  --learning_rate=2e-5 \
                  --seed=42

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:3.39 - acc1:0.51 - acc2:0.48:  92% 317/344 [18:24<01:05,  2.42s/it]Traceback (most recent call last):
  File "main_tpu.py", line 389, in <module>
  File "main_tpu.py", line 386, in main
    xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 296, in spawn
    start_method=start_method)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/spawn.py", line 158, in start_processes
    while not context.join():
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/spawn.py", line 78, in join
    timeout=timeout,
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 911, in wait

In [0]:
!python main_tpu.py --fold_index=2 \
                  --model_path="albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=1001

Training is Starting ...... 
Train E:1 - Loss:3.23 - acc1:0.54 - acc2:0.52: 100% 86/86 [03:36<00:00,  2.51s/it]
Eval E:1 - Loss:2.23 - Jac:0.64: 100% 22/22 [00:50<00:00,  2.29s/it]
**** Epoch 1 **==>** Jaccard = 0.6443821466857951
**** Model Improved !!!! Saving Model
Train E:2 - Loss:2.22 - acc1:0.61 - acc2:0.63:   8% 7/86 [00:19<03:30,  2.67s/it]Traceback (most recent call last):
  File "main_tpu.py", line 389, in <module>
  File "main_tpu.py", line 386, in main
    xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 296, in spawn
    start_method=start_method)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/spawn.py", line 158, in start_processes
    while not context.join():
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/spawn.py", line 78, in join
    timeout=timeout,
  File "/usr/lib/python3.6/multiprocessing/connection.py", lin

In [0]:
# albert-base-v1 - 0.6941892598602024
# albert-base-v2 - 0.6884260813070375
# elgeish/cs224n-squad2.0-albert-base-v2 - 0.694456474487533
# elgeish/cs224n-squad2.0-albert-large-v2 - 0.6991612173425853
# albert-large-v2 - 0.696910668336368

In [0]:
import transformers

In [9]:
tokenizer = transformers.AlbertTokenizer.from_pretrained("elgeish/cs224n-squad2.0-albert-large-v2")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [0]:
!mkdir cs224n-squad2.0-albert-large-v2

In [15]:
tokenizer.save_pretrained("cs224n-squad2.0-albert-large-v2/")

('cs224n-squad2.0-albert-large-v2/spiece.model',
 'cs224n-squad2.0-albert-large-v2/special_tokens_map.json',
 'cs224n-squad2.0-albert-large-v2/added_tokens.json')

In [0]:
!python main_tpu.py --fold_index=1 \
                  --model_path="albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=123 \
                  --sp_path='cs224n-squad2.0-albert-large-v2/spiece.model'

Training is Starting ...... 
Train E:1 - Loss:9.06 - acc1:0.06 - acc2:0.04: 100% 86/86 [03:38<00:00,  2.54s/it]
Eval E:1 - Loss:10.51 - Jac:0.42: 100% 22/22 [00:50<00:00,  2.31s/it]
**** Epoch 1 **==>** Jaccard = 0.41806439717462485
**** Model Improved !!!! Saving Model
Train E:2 - Loss:10.53 - acc1:0.01 - acc2:0.00: 100% 86/86 [03:31<00:00,  2.46s/it]
Eval E:2 - Loss:10.51 - Jac:0.42: 100% 22/22 [00:45<00:00,  2.07s/it]
**** Epoch 2 **==>** Jaccard = 0.41840916106834597
**** Model Improved !!!! Saving Model
Train E:3 - Loss:10.53 - acc1:0.00 - acc2:0.01: 100% 86/86 [03:29<00:00,  2.44s/it]
Eval E:3 - Loss:10.51 - Jac:0.42: 100% 22/22 [00:44<00:00,  2.04s/it]
**** Epoch 3 **==>** Jaccard = 0.41644347682328864
Train E:4 - Loss:10.53 - acc1:0.01 - acc2:0.01:  69% 59/86 [02:25<01:05,  2.43s/it]

In [17]:
!python main_tpu.py --fold_index=1 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42 \
                  --sp_path='cs224n-squad2.0-albert-large-v2/spiece.model'

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 665, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py", line 30, in <module>
    from tensorflow.python.ops import weights_broadcast_ops
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/weights_broadcast_ops.py", line 30, in <module>
    from tensorflow.python.ops import sets
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/sets.py", line 23, in <module>
    from tensorflow.python.ops.sets_impl import *
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/sets_impl.py", line 25, in <module>
    from tensorflow.python.ops import gen_set_ops
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gen_set_ops.py", line 294, in <modul

In [18]:
!python main_tpu.py --fold_index=2 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42 \
                  --sp_path='cs224n-squad2.0-albert-large-v2/spiece.model'

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:5.78 - acc1:0.33 - acc2:0.21: 100% 86/86 [05:05<00:00,  3.55s/it]
Eval E:1 - Loss:4.54 - Jac:0.56: 100% 22/22 [02:13<00:00,  6.07s/it]
**** Epoch 1 **==>** Jaccard = 0.5595298441872869
**** Model Improved !!!! Saving Model
Train E:2 - Loss:4.37 - acc1:0.39 - acc2:0.26: 100% 86/86 [03:34<00:00,  2.49s/it]
Eval E:2 - Loss:4.37 - Jac:0.55: 100% 22/22 [00:45<00:00,  2.08s/it]
**** Epoch 2 **==>** Jaccard = 0.551285265538831
Train E:3 - Loss:4.14 - acc1:0.42 - acc2:0.27: 100% 86/86 [03:34<00:00,  2.49s/it]
Eval E:3 - Loss:4.34 - Jac:0.57: 100% 22/22 [00:45<00:00,  2.08s/it]
**** Epoch 3 **==>** Jaccard = 0.5657832114358907
**** Model Improved !!!! Saving Model
Train E:4 - Loss:4.13 - acc1:0.41 - acc2:0.27:   7% 6/86 [00:17<03:

KeyboardInterrupt: ignored

In [0]:
!python main_tpu.py --fold_index=3 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42 \
                  --sp_path='cs224n-squad2.0-albert-large-v2/spiece.model'

In [0]:
!python main_tpu.py --fold_index=4 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42 \
                  --sp_path='cs224n-squad2.0-albert-large-v2/spiece.model'

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42

Downloading: 100% 700/700 [00:00<00:00, 506kB/s]
Downloading: 100% 760k/760k [00:00<00:00, 4.95MB/s]
Downloading: 100% 156/156 [00:00<00:00, 89.6kB/s]
Downloading: 100% 39.0/39.0 [00:00<00:00, 25.1kB/s]
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Downloading: 100% 70.8M/70.8M [00:01<00:00, 54.1MB/s]
Training is Starting ...... 
Train E:1 - Loss:2.92 - acc1:0.58 - acc2:0.60: 100% 86/86 [06:35<00:00,  4.60s/it]
Eval E:1 - Loss:1.83 - Jac:0.68: 100% 22/22 [02:13<00:00,  6.05s/it]
**** Epoch 1 **==>** Jaccard = 0.6848965801234421
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.73 - acc1:0.70 - acc2:0.72: 100% 86/86 [03:26<00:00,  2.40s/it]
Eval E:2 - Loss:1.73 - Jac:0.69: 100% 22/22 [00:44<00:00,  2.03s/it]
**** Epoch 2 **==>** Jaccard = 0.6914089615658042
**** Model Improved !!!! Saving Model
Train E:3 -

In [0]:
!python main_tpu.py --fold_index=1 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:2.61 - acc1:0.60 - acc2:0.63: 100% 86/86 [06:37<00:00,  4.62s/it]
Eval E:1 - Loss:1.78 - Jac:0.70: 100% 22/22 [02:14<00:00,  6.11s/it]
**** Epoch 1 **==>** Jaccard = 0.6990274843297151
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.68 - acc1:0.70 - acc2:0.72: 100% 86/86 [03:25<00:00,  2.38s/it]
Eval E:2 - Loss:1.71 - Jac:0.70: 100% 22/22 [00:44<00:00,  2.03s/it]
**** Epoch 2 **==>** Jaccard = 0.7036393548913482
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.51 - acc1:0.72 - acc2:0.74: 100% 86/86 [03:25<00:00,  2.39s/it]
Eval E:3 - Loss:1.79 - Jac:0.69: 100% 22/22 [00:43<00:00,  1.99s/it]
**** Epoch 3 **==>** Jaccard = 0.6925826828435222
Train E:4 - Loss:1.37 - acc1:0.74 - acc2:0.76: 100% 86/86 [03:26<0

In [0]:
!python main_tpu.py \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=1001

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:2.78 - acc1:0.59 - acc2:0.60: 100% 86/86 [03:40<00:00,  2.57s/it]
Eval E:1 - Loss:1.82 - Jac:0.69: 100% 22/22 [00:50<00:00,  2.30s/it]
**** Epoch 1 **==>** Jaccard = 0.6857064023082711
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.71 - acc1:0.70 - acc2:0.71: 100% 86/86 [03:27<00:00,  2.41s/it]
Eval E:2 - Loss:1.70 - Jac:0.70: 100% 22/22 [00:45<00:00,  2.06s/it]
**** Epoch 2 **==>** Jaccard = 0.7012697796538987
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.53 - acc1:0.72 - acc2:0.74: 100% 86/86 [03:29<00:00,  2.44s/it]
Eval E:3 - Loss:1.71 - Jac:0.70: 100% 22/22 [00:44<00:00,  2.01s/it]
**** Epoch 3 **==>** Jaccard = 0.7028332198531768
**** Model Improved !!!! Saving Model
Train E:4 - Loss:1.37 - acc1

In [0]:
!python main_tpu.py --fold_index=3 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:2.54 - acc1:0.61 - acc2:0.63: 100% 86/86 [05:03<00:00,  3.52s/it]
Eval E:1 - Loss:1.79 - Jac:0.69: 100% 22/22 [00:51<00:00,  2.32s/it]
**** Epoch 1 **==>** Jaccard = 0.6902198294760687
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.67 - acc1:0.70 - acc2:0.72: 100% 86/86 [03:26<00:00,  2.40s/it]
Eval E:2 - Loss:1.74 - Jac:0.69: 100% 22/22 [00:44<00:00,  2.04s/it]
**** Epoch 2 **==>** Jaccard = 0.694721206318744
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.53 - acc1:0.72 - acc2:0.74: 100% 86/86 [03:26<00:00,  2.40s/it]
Eval E:3 - Loss:1.74 - Jac:0.70: 100% 22/22 [00:44<00:00,  2.01s/it]
**** Epoch 3 **==>** Jaccard = 0.6966806989867863
**** Model Improved !!!! Saving Model
Train E:4 - Loss:1.40 - acc1:

In [0]:
!python main_tpu.py --fold_index=4 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=42

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:2.37 - acc1:0.63 - acc2:0.65: 100% 86/86 [03:41<00:00,  2.58s/it]
Eval E:1 - Loss:1.74 - Jac:0.70: 100% 22/22 [00:50<00:00,  2.30s/it]
**** Epoch 1 **==>** Jaccard = 0.7013956327429918
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.63 - acc1:0.70 - acc2:0.72: 100% 86/86 [03:26<00:00,  2.40s/it]
Eval E:2 - Loss:1.71 - Jac:0.71: 100% 22/22 [00:44<00:00,  2.03s/it]
**** Epoch 2 **==>** Jaccard = 0.7051919533714295
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.46 - acc1:0.73 - acc2:0.75: 100% 86/86 [03:25<00:00,  2.39s/it]
Eval E:3 - Loss:1.80 - Jac:0.70: 100% 22/22 [00:44<00:00,  2.04s/it]
**** Epoch 3 **==>** Jaccard = 0.7046383712124276
Train E:4 - Loss:1.29 - acc1:0.75 - acc2:0.77: 100% 86/86 [03:26<0

In [0]:
import pandas as pd

In [0]:
train_df = pd.read_csv(f"../input/train_5folds_seed{42}.csv")

In [0]:
!python main_tpu.py --fold_index=0 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2_seed123" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=123

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:2.39 - acc1:0.63 - acc2:0.64: 100% 86/86 [03:54<00:00,  2.72s/it]
Eval E:1 - Loss:1.73 - Jac:0.70: 100% 22/22 [00:51<00:00,  2.32s/it]
**** Epoch 1 **==>** Jaccard = 0.696748076046493
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.60 - acc1:0.71 - acc2:0.72: 100% 86/86 [03:27<00:00,  2.41s/it]
Eval E:2 - Loss:1.71 - Jac:0.70: 100% 22/22 [00:44<00:00,  2.03s/it]
**** Epoch 2 **==>** Jaccard = 0.7005536886356298
**** Model Improved !!!! Saving Model
Train E:3 - Loss:1.38 - acc1:0.74 - acc2:0.76: 100% 86/86 [03:26<00:00,  2.40s/it]
Eval E:3 - Loss:1.88 - Jac:0.69: 100% 22/22 [00:44<00:00,  2.01s/it]
**** Epoch 3 **==>** Jaccard = 0.6941113960222325
Train E:4 - Loss:1.15 - acc1:0.78 - acc2:0.80: 100% 86/86 [03:27<00

In [0]:
!python main_tpu.py --fold_index=1 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2_seed123" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=123

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:8.02 - acc1:0.17 - acc2:0.17: 100% 86/86 [03:36<00:00,  2.52s/it]
Eval E:1 - Loss:10.51 - Jac:0.42: 100% 22/22 [00:51<00:00,  2.34s/it]
**** Epoch 1 **==>** Jaccard = 0.41642368551996883
**** Model Improved !!!! Saving Model
Train E:2 - Loss:10.53 - acc1:0.00 - acc2:0.01: 100% 86/86 [03:25<00:00,  2.39s/it]
Eval E:2 - Loss:10.51 - Jac:0.41: 100% 22/22 [00:44<00:00,  2.03s/it]
**** Epoch 2 **==>** Jaccard = 0.41465905704412276
Train E:3 - Loss:10.53 - acc1:0.01 - acc2:0.01:  24% 21/86 [00:52<02:43,  2.51s/it]Traceback (most recent call last):

Error in atexit._run_exitfuncs:
^C


In [0]:
!python main_tpu.py --fold_index=2 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2_seed123" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=123

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:4.84 - acc1:0.39 - acc2:0.40: 100% 86/86 [03:39<00:00,  2.55s/it]
Eval E:1 - Loss:4.08 - Jac:0.58: 100% 22/22 [00:51<00:00,  2.32s/it]
**** Epoch 1 **==>** Jaccard = 0.5814317578799085
**** Model Improved !!!! Saving Model
Train E:2 - Loss:3.42 - acc1:0.57 - acc2:0.57: 100% 86/86 [03:27<00:00,  2.41s/it]
Eval E:2 - Loss:3.34 - Jac:0.58: 100% 22/22 [00:44<00:00,  2.04s/it]
**** Epoch 2 **==>** Jaccard = 0.5820424981000631
**** Model Improved !!!! Saving Model
Train E:3 - Loss:4.72 - acc1:0.54 - acc2:0.51: 100% 86/86 [03:27<00:00,  2.41s/it]
Eval E:3 - Loss:4.21 - Jac:0.58: 100% 22/22 [00:44<00:00,  2.00s/it]
**** Epoch 3 **==>** Jaccard = 0.5762127037251281
Train E:4 - Loss:3.92 - acc1:0.56 - acc2:0.56: 100% 86/86 [03:28<0

In [0]:
!python main_tpu.py --fold_index=3 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2_seed123" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=123

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:10.38 - acc1:0.01 - acc2:0.01: 100% 86/86 [04:53<00:00,  3.42s/it]
Eval E:1 - Loss:10.51 - Jac:0.41: 100% 22/22 [00:49<00:00,  2.26s/it]
**** Epoch 1 **==>** Jaccard = 0.4146891431319952
**** Model Improved !!!! Saving Model
Train E:2 - Loss:10.52 - acc1:0.01 - acc2:0.01: 100% 86/86 [03:25<00:00,  2.39s/it]
Eval E:2 - Loss:10.51 - Jac:0.42: 100% 22/22 [00:45<00:00,  2.05s/it]
**** Epoch 2 **==>** Jaccard = 0.4152659476489986
**** Model Improved !!!! Saving Model
Train E:3 - Loss:10.52 - acc1:0.01 - acc2:0.01: 100% 86/86 [03:28<00:00,  2.42s/it]
Eval E:3 - Loss:10.51 - Jac:0.42: 100% 22/22 [00:44<00:00,  2.02s/it]
**** Epoch 3 **==>** Jaccard = 0.4165461052661548
**** Model Improved !!!! Saving Model
Train E:4 - Loss:10.52

In [0]:
!python main_tpu.py --fold_index=4 \
                  --model_path="elgeish/cs224n-squad2.0-albert-large-v2" \
                  --output_dir="../weights/albert-large" \
                  --exp_name="elgeish/cs224n-squad2.0-albert-large-v2_seed123" \
                  --batch_size=32 \
                  --learning_rate=2e-5 \
                  --seed=123

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
Training is Starting ...... 
Train E:1 - Loss:2.41 - acc1:0.63 - acc2:0.64: 100% 86/86 [03:43<00:00,  2.59s/it]
Eval E:1 - Loss:1.73 - Jac:0.70: 100% 22/22 [00:50<00:00,  2.28s/it]
**** Epoch 1 **==>** Jaccard = 0.6979979055893206
**** Model Improved !!!! Saving Model
Train E:2 - Loss:1.62 - acc1:0.71 - acc2:0.72: 100% 86/86 [03:30<00:00,  2.44s/it]
Eval E:2 - Loss:1.70 - Jac:0.69: 100% 22/22 [00:45<00:00,  2.05s/it]
**** Epoch 2 **==>** Jaccard = 0.6924751796580901
Train E:3 - Loss:1.43 - acc1:0.73 - acc2:0.75: 100% 86/86 [03:28<00:00,  2.43s/it]
Eval E:3 - Loss:1.78 - Jac:0.70: 100% 22/22 [00:44<00:00,  2.02s/it]
**** Epoch 3 **==>** Jaccard = 0.699535748288139
**** Model Improved !!!! Saving Model
Train E:4 - Loss:1.32 - acc1:0.74 - acc2:0.77:  13% 11/86 [00:28<03